#### Loading Star Catalogs
Catalogs can be downloaded from a variety of services like VizieR, typically consist of a data file and a ReadMe, explaining its formatting. Below, I will discuss a couple of ways to load these data files and more optimal ways to store them.

We'll start with basic imports, and the `astropy` provided method of loading. 

In [2]:
from astropy.io import ascii
from astropy.table import Table, vstack
import numpy as np
import time
import numpy as np
import pandas as pd

In [ ]:
# Uses Astropy to read the Tycho catalog
### LARGE catalog, and its currently unoptimized, so it will take awhile to load. 
#tycho = ascii.read('./catalog/tyc_main.dat', readme="./catalog/ReadMe")
#print(f"Loaded {len(tycho)} Tycho objects")
#print(tycho.colnames)

Astropy also provides a method for loading catalog into a table, which is slightly faster and more efficient.

In [ ]:
tycho = Table.read('./catalog/tyc_main.dat', format='ascii', readme="./catalog/ReadMe")

From a `Table`, we can save it directly to a `.fits` file format, which is **MUCH** faster to load. 
Beforehand, we can remove unncessary columns to also increase speed. 

In [ ]:
col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'HD']
tycho = tycho[col_names]
tycho.write('./catalog/tyc_main.fits', format='fits', overwrite=True)

For sky navigation, we'd like some identification for useful, bright stars. 
We will add an extra `Name` column to our fits data file.
We will use constellation-based names like "33 Psc" as well as coloquial names like Rigel or Polaris.

In [3]:
tycho = Table.read('./catalog/tyc_main.fits')
common_names = Table.read("./catalog/names.fits")
bright_stars = Table.read('./catalog/catalog', format='ascii', readme="./catalog/ReadMe_bright")

Loop through every entry in Tycho, and look for a name in the other two catalogs by HD number id.

In [9]:
# create name column in tycho
tycho['Name'] = np.full(len(tycho), '', dtype='U50')
# loop through tycho catalog. look for HD number in common names, then bright stars
for i, row in enumerate(tycho):
    hd_number = row['HD']
    if hd_number == 0:
        continue

    name = ""
    # Check if HD number exists in common names or bright stars
    if hd_number in common_names['HD']:
        common = common_names[common_names['HD'] == hd_number]['name'][0]
        common = str(common).replace('--', '').strip()
        if len(common) > 0:
            name = common
    elif hd_number in bright_stars['HD']:
        bright = bright_stars[bright_stars['HD'] == hd_number]['Name'][0]
        bright = str(bright).replace('--', '').strip()
        if len(bright) > 0:
            name = bright
    tycho['Name'][i] = name
    if len(name) > 0:
        print(f"Found name for HD {hd_number}: {name}")

# Save the updated tycho table with names
tycho.write('./catalog/tyc.fits', format='fits', overwrite=True)

Found name for HD 2913: 51    Psc
Found name for HD 4526: 60    Psc
Found name for HD 4628: 96 G  PSC
Found name for HD 4627: 62    Psc
Found name for HD 7014: 33    Cet
Found name for HD 6479: 77    Psc
Found name for HD 6480: 77    Psc
Found name for HD 6386: 73    Psc
Found name for HD 6763: 80    Psc
Found name for HD 7446: 88    Psc
Found name for HD 9138: 98Mu  Psc
Found name for HD 12573: 60    Cet
Found name for HD 14652: 69    Cet
Found name for HD 12235: 112    Psc
Found name for HD 12447: 113Alp Psc
Found name for HD 16970: 86Gam Cet
Found name for HD 18883: 93    Cet
Found name for HD 18884: MENKAR
Found name for HD 20791: 97Kap2Cet
Found name for HD 22796: 12    Tau
Found name for HD 24263: 31    Tau
Found name for HD 25558: 40    Tau
Found name for HD 26462: 45    Tau
Found name for HD 31767: 10Pi 6Ori
Found name for HD 31139: 5    Ori
Found name for HD 35439: 25Psi1Ori
Found name for HD 33856: 17Rho Ori
Found name for HD 34658: 21    Ori
Found name for HD 35149: 23    Or

Now that our catalog has names, lets remove unnecessary columns so loading and searching is faster.

In [11]:
tycho = Table.read('./catalog/tyc.fits')
col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'Name']
tycho = tycho[col_names]
tycho.write('./catalog/tyc.fits', format='fits', overwrite=True)

Lets see whats in our catalog now.

In [116]:
tycho = Table.read('./catalog/tyc.fits')
# get brighest 10 stars
brightest = tycho[np.argsort(tycho['Vmag'])[:100]]
print("Brightest 10 stars:")
for star in brightest:
    print(f"TYC: {star['TYC']}, Vmag: {star['Vmag']}, Name: {star['Name']}")

Brightest 10 stars:
TYC: 5949  2777 1, Vmag: -1.44, Name: SIRIUS
TYC: 8534  2277 1, Vmag: -0.63, Name: CANOPUS
TYC: 9007  5849 1, Vmag: -0.01, Name: RIGEL KENTAURUS
TYC: 3105  2070 1, Vmag: 0.03, Name: VEGA
TYC: 3358  3141 1, Vmag: 0.08, Name: CAPELLA
TYC: 1472  1436 1, Vmag: 0.16, Name: ARCTURUS
TYC: 5331  1752 1, Vmag: 0.28, Name: RIGEL
TYC: 187  2184 1, Vmag: 0.4, Name: PROCYON
TYC: 8478  1395 1, Vmag: 0.54, Name: ACHERNAR
TYC: 129  1873 1, Vmag: 0.57, Name: BETELGEUSE
TYC: 9005  3919 1, Vmag: 0.64, Name: --
TYC: 1058  3399 1, Vmag: 0.93, Name: ALTAIR
TYC: 1266  1416 1, Vmag: 0.99, Name: ALDEBARAN
TYC: 5547  1518 1, Vmag: 1.06, Name: SPICA
TYC: 6803  2158 1, Vmag: 1.07, Name: ANTARES
TYC: 1920  2194 1, Vmag: 1.22, Name: POLLUX
TYC: 6977  1267 1, Vmag: 1.23, Name: FOMALHAUT
TYC: 8979  3464 1, Vmag: 1.28, Name: ACRUX
TYC: 8659  3107 1, Vmag: 1.31, Name: --
TYC: 3574  3347 1, Vmag: 1.33, Name: DENEB
TYC: 9007  5848 1, Vmag: 1.35, Name: Alp2Cen
TYC: 833  1381 1, Vmag: 1.41, Name: REGULU

We are still missing names for some of our brightest stars, lets use another catalog, Hipparcos. 
For every Tycho entry above a threshold brightness and without a HD number, we will look for its TYC ID in Hipparcos to find a correct HD number. We can then research for a name in common names and bright stars. 

In [114]:
tycho = Table.read('./catalog/tyc_main.fits')
hipparcos = Table.read("./catalog/hip_main.dat", format='ascii', readme="./catalog/ReadMe_hip")

col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'HD', 'HIP']
tycho = tycho[col_names]

# Filter Vmag < 6.0 and no HD number
#tycho_short = tycho[(tycho['Vmag'] < 6.0) & (tycho['Name'].mask)]

# create name column in tycho short
tycho['Name'] = np.full(len(tycho), '', dtype='U50')
# loop through tycho catalog. look for HD number in common names, then bright stars
for i, row in enumerate(tycho):
    name = ""
    hd_number = row['HD']
    mag = row['Vmag']
    if mag > 8.0:
        continue
    if not isinstance(hd_number, np.int64):
        hip_number = row['HIP']

        hip = hipparcos[hipparcos['HIP'] == hip_number]
        if len(hip) == 0:
            continue
        else:
            hd_number = hip['HD'][0]
    
    if hd_number in common_names['HD']:
            common = common_names[common_names['HD'] == hd_number]['name'][0]
            common = str(common).replace('--', '').strip()
            if len(common) > 0:
                name = common
    elif hd_number in bright_stars['HD']:
        bright = bright_stars[bright_stars['HD'] == hd_number]['Name'][0]
        bright = str(bright).replace('--', '').strip()
        if len(bright) > 0:
            name = bright

    
    tycho['Name'][i] = name
    if len(name) > 0:
        print(f"Found{name}")

col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'Name']
tycho = tycho[col_names]
tycho.write('./catalog/tyc.fits', format='fits', overwrite=True)

Found51    Psc
Found60    Psc
Found96 G  PSC
Found62    Psc
Found33    Cet
Found77    Psc
Found77    Psc
Found73    Psc
Found80    Psc
Found88    Psc
Found98Mu  Psc
Found60    Cet
Found69    Cet
Found112    Psc
Found113Alp Psc
Found113Alp Psc
Found86Gam Cet
Found86Gam Cet
Found268 G CET
Found93    Cet
FoundMENKAR
Found97Kap2Cet
Found12    Tau
Found31    Tau
Found40    Tau
Found45    Tau
Found10Pi 6Ori
Found5    Ori
Found25Psi1Ori
Found17Rho Ori
Found21    Ori
Found23    Ori
Found30Psi2Ori
Found30Psi2Ori
FoundBELLATRIX
Found51    Ori
Found56    Ori
Found59    Ori
Found33    Ori
Found33    Ori
Found38    Ori
Found47Ome Ori
Found32    Ori
Found52    Ori
FoundBETELGEUSE
Found63    Ori
Found12    Mon
Found5Eta CMi
Found9Del3CMi
Found7Del1CMi
Found8Del2CMi
Found14    CMi
FoundPROCYON
Found7Eta Hya
Found10    Hya
Found11Eps Hya
Found11Eps Hya
Found13Rho Hya
Found18Ome Hya
Found7    Sex
Found4    Sex
Found10    Leo
FoundRS SEX
Found13    Sex
Found19    Sex
Found14    Sex
Found43    Leo
Found36

We can now add methods for searching directly for stars by name or coordinates.

In [38]:
tycho = Table.read('../data/tyc.fits')

def clean_name(n) -> str:
    if n is np.ma.masked:
        return ''
    n = str(n)
    n = n.strip()
    return n.lower()

def search_by_name(n: str):
    n = clean_name(n)
    name_col = tycho['Name'].filled('')
    name_strings = np.array([clean_name(n) for n in name_col])

    results = tycho[name_strings == n]
    print(len(results))

    return results

def search_by_coordinate(ra: float, dec: float, radius: float = 0.1):
    ra = float(ra)
    dec = float(dec)
    radius = float(radius)

    # Convert radius from degrees to radians
    radius_rad = np.radians(radius)

    # Calculate the distance in degrees
    delta_ra = np.radians(tycho['RAdeg'] - ra)
    delta_dec = np.radians(tycho['DEdeg'] - dec)

    # Haversine formula to calculate distance
    a = (np.sin(delta_dec / 2) ** 2 +
         np.cos(np.radians(dec)) * np.cos(np.radians(tycho['DEdeg'])) *
         np.sin(delta_ra / 2) ** 2)
    c = 2 * np.arcsin(np.sqrt(a))

    # Distance in radians
    distance_rad = c

    # Filter results within the specified radius
    results = tycho[distance_rad <= radius_rad]
    
    return results

target = search_by_name("pleiades")
print(f"{target}")

near = search_by_coordinate(target['RAdeg'], target['DEdeg'])
print(near)



1
RAdeg DEdeg  TYC Vmag   Name  
 deg   deg       mag          
----- ------ --- ---- --------
3.783 24.117  OC  1.2 Pleiades
  RAdeg       DEdeg        TYC      Vmag   Name  
   deg         deg                  mag          
---------- ----------- ------------ ---- --------
3.86066522 24.06974512 1730  1332 1 9.47       --
     3.783      24.117           OC  1.2 Pleiades


C:\Users\314ow\AppData\Local\Temp\ipykernel_28424\2143646842.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ra = float(ra)
C:\Users\314ow\AppData\Local\Temp\ipykernel_28424\2143646842.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dec = float(dec)


In [34]:
messier = pd.read_csv('./catalog/messier.csv')
ngc = pd.read_csv('./catalog/ngc.csv')

dsos = []

for m in messier.itertuples():
    name = m.comments
    if name is np.nan:
        name = f"M-{m.messier_id}"
    ra = m.ra
    dec = m.dec
    vmag = m.v_mag
    dso_type = m.type
    dsos.append((ra, dec, dso_type, vmag, name))

for n in ngc.itertuples():
    name = n.comments
    if name is np.nan:
        name = f"NGC-{n.ngc_id}"
    ra = n.ra
    dec = n.dec
    vmag = n.v_mag
    dso_type = m.type
    dsos.append((ra, dec, dso_type, vmag, name))

tycho = Table.read('../data/tyc.fits')
tycho = vstack([tycho, Table(rows=dsos, names=('RAdeg', 'DEdeg', 'TYC', 'Vmag', 'Name'))])
tycho.write('../data/tyc.fits', format='fits', overwrite=True)

In [5]:
tycho = Table.read('../data/tyc.fits')
mask = [len(t['TYC']) == 2 for t in tycho]
print(tycho[mask])
print(len(tycho[mask]))

RAdeg   DEdeg  TYC Vmag                        Name                       
 deg     deg       mag                                                    
------ ------- --- ---- --------------------------------------------------
 3.783  24.117  OC  1.2                                           Pleiades
 8.667  19.667  OC  3.1 Beehive cluster.  358 stars approx. 500 ly distant
17.883 -34.793  OC  3.3                                                M-7
   0.7  41.267  SG  3.4                                   Andromeda galaxy
 5.583  -5.383  DN  4.0                                 Great Orion nebula
17.667 -32.253  OC  4.2                                  Butterfly cluster
   7.6 -14.483  OC  4.4                                               M-47
 6.767 -20.757  OC  4.5                                               M-41
 18.28  -18.55  OC  4.5                                               M-24
   ...     ... ...  ...                                                ...
 17.36   75.85  PN   --  